<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Word2vect


In [1]:
# %load_ext lab_black

import numpy as np
import pandas as pd
import time

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(
    ["que dia es hoy", "martes el dia de hoy es martes", "martes muchas gracias"]
)

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

#### Resolución

Para obtener los términos lo que hacemos es unir todos los documentos en un única cadena de texto, luego se dividen las palabras por espacios en blanco, es decir, se crea un vector donde cada elemento es una palabra. Finalmente podemos utilizar *np.unique()* o *set()* para eliminar los elementos duplicados. Probamos ambos métodos y evaluamos al performance de cada uno.

In [4]:
methods = [("np.unique()", np.unique), ("set()", set)]

for method in methods:
    print(f"Vocabulario del corpus utilizando {method[0]}:")
    start = time.time()
    vocab = method[1](" ".join(corpus).split())
    end = time.time()
    for word in vocab:
        print(f"- {word}")
    print(f"Tiempo empleado en la conversion: {end - start}")
    print()

Vocabulario del corpus utilizando np.unique():
- de
- dia
- el
- es
- gracias
- hoy
- martes
- muchas
- que
Tiempo empleado en la conversion: 0.00011229515075683594

Vocabulario del corpus utilizando set():
- que
- el
- muchas
- es
- gracias
- de
- martes
- hoy
- dia
Tiempo empleado en la conversion: 8.106231689453125e-06



Vemos que para el caso de string el tiempo empleado es menor al usar *set()*.

In [5]:
# Definimos una función que va a ser util luego
calc_vocabulary = lambda corpus: set(" ".join(corpus).split())

### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

#### Resolución

Para hacer one-hot encoding podemos iterar sobre todos los documentos del corpus y a la vez sobre todo el vocabulario, de esta manera generamos una matriz con 1 cuando un término se encuentre en le documento o un 0 cuando no.

In [6]:
def calc_one_hot(corpus):
    vocab = calc_vocabulary(corpus)
    one_hot_matrix = []

    for doc in corpus:
        row = []
        splitted_doc = doc.split()
        for word in vocab:
            row.append(1 if word in splitted_doc else 0)
        one_hot_matrix.append(row)

    return np.array(one_hot_matrix)


one_hot_matrix = calc_one_hot(corpus)

# Creamos un DataFrame de pandas para poder visualizar fácilmente los resultados
pd.DataFrame(one_hot_matrix, columns=vocab, index=corpus)

,que,el,muchas,es,gracias,de,martes,hoy,dia
que dia es hoy,1,0,0,1,0,0,0,1,1
martes el dia de hoy es martes,0,1,0,1,0,1,1,1,1
martes muchas gracias,0,0,1,0,1,0,1,0,0


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

#### Resolución

El método de resolución el similar al one-hot encoding, la diferencia es que en este caso debemos contar la cantidad de ocurrencias de cada palabra en el documento.

In [7]:
def calc_frec(corpus):
    vocab = calc_vocabulary(corpus)
    frec_matrix = []

    for doc in corpus:
        row = []
        splitted_doc = doc.split()
        for word in vocab:
            row.append(splitted_doc.count(word))
        frec_matrix.append(row)

    return np.array(frec_matrix)


frec_matrix = calc_frec(corpus)

pd.DataFrame(frec_matrix, columns=vocab, index=corpus)

,que,el,muchas,es,gracias,de,martes,hoy,dia
que dia es hoy,1,0,0,1,0,0,0,1,1
martes el dia de hoy es martes,0,1,0,1,0,1,2,1,1
martes muchas gracias,0,0,1,0,1,0,1,0,0


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

#### Resolución

- TF es igual a la matriz de frecuencias calculada en el punto anterior.

- DF se puede calcular sumando la mátriz de one-hot encondig en *axis=0* (verticalmente).

- La fórmula de IDF es la siguiente:

  $IDF = log_{10}(\frac{n \ docs}{DF})$


- TF-IDF se calcula como:

  $TFIDF = TF\times IDF$

In [8]:
def calc_idf(corpus):
    df = calc_one_hot(corpus).sum(axis=0)
    return np.log10(corpus.shape[0] / df).reshape(1, -1)


idf = calc_idf(corpus)

print("IDF")
pd.DataFrame(idf, columns=vocab, index=["IDF"])

IDF


,que,el,muchas,es,gracias,de,martes,hoy,dia
IDF,0.477121,0.477121,0.477121,0.176091,0.477121,0.477121,0.176091,0.176091,0.176091


In [9]:
tf_idf = frec_matrix * idf

print("TF-IDF")
pd.DataFrame(tf_idf, columns=vocab, index=corpus)

TF-IDF


,que,el,muchas,es,gracias,de,martes,hoy,dia
que dia es hoy,0.477121,0.000000,0.000000,0.176091,0.000000,0.000000,0.000000,0.176091,0.176091
martes el dia de hoy es martes,0.000000,0.477121,0.000000,0.176091,0.000000,0.477121,0.352183,0.176091,0.176091
martes muchas gracias,0.000000,0.000000,0.477121,0.000000,0.477121,0.000000,0.176091,0.000000,0.000000


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

#### Resolución

La función utilizará la fórmula mostrada en el punto anterior para calcular TF-IDF, luego se calcula la similutd del coseno para todos los elementos del corpus (excepto para el seleccionado) y se ordenan los resultados obtenidos de mayor a menor, es decir de más a menos parecidos.

In [10]:
def calc_cosine_similarity(corpus, idx):
    tf_idf = calc_frec(corpus) * calc_idf(corpus)
    scores = []
    for i, doc in enumerate(tf_idf):
        if i != idx:
            scores.append((i, cosine_similarity(tf_idf[idx], doc)))
    scores = list(scores)
    scores.sort(key=lambda x: x[1], reverse=True)

    return scores


print("Corpus:")
for i, doc in enumerate(corpus):
    print(f"({i + 1}) {doc}")

print("\nCosine similarity:")
for i, doc in enumerate(corpus):
    similarity = calc_cosine_similarity(corpus, i)
    for item in similarity:
        print(f"({i + 1}) con ({item[0] + 1}) {(item[1] * 100):.2f}%")

Corpus:
(1) que dia es hoy
(2) martes el dia de hoy es martes
(3) martes muchas gracias

Cosine similarity:
(1) con (2) 20.03%
(1) con (3) 0.00%
(2) con (1) 20.03%
(2) con (3) 10.85%
(3) con (2) 10.85%
(3) con (1) 0.00%
